In [182]:
import pandas as pd
df1 = pd.read_csv('kdrama.csv')
df1.head()

,Name,Aired Date,Year of release,Original Network,Aired On,Number of Episodes,Duration,Content Rating,Rating,Synopsis,Genre,Tags,Director,Screenwriter,Cast,Production companies,Rank
0,Move to Heaven,"May 14, 2021",2021,Netflix,Friday,10,52 min.,18+ Restricted (violence & profanity),9.2,Geu Roo is a young autistic man. He works for ...,"Life, Drama, Family","Autism, Uncle-Nephew Relationship, Death, Sava...",Kim Sung Ho,Yoon Ji Ryun,"Lee Je Hoon, Tang Jun Sang, Hong Seung Hee, Ju...","Page One Film, Number Three Pictures",#1
1,Flower of Evil,"Jul 29, 2020 - Sep 23, 2020",2020,tvN,"Wednesday, Thursday",16,1 hr. 10 min.,15+ - Teens 15 or older,9.1,Although Baek Hee Sung is hiding a dark secret...,"Thriller, Romance, Crime, Melodrama","Married Couple, Deception, Suspense, Family Se...","Kim Chul Gyu, Yoon Jong Ho",Yoo Jung Hee,"Lee Joon Gi, Moon Chae Won, Jang Hee Jin, Seo ...",Monster Union,#2
2,Hospital Playlist,"Mar 12, 2020 - May 28, 2020",2020,"Netflix, tvN",Thursday,12,1 hr. 30 min.,15+ - Teens 15 or older,9.1,The stories of people going through their days...,"Friendship, Romance, Life, Medical","Strong Friendship, Multiple Mains, Best Friend...",Shin Won Ho,Lee Woo Jung,"Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Kim...","Egg Is Coming, CJ ENM",#3
3,Hospital Playlist 2,"Jun 17, 2021 - Sep 16, 2021",2021,"Netflix, tvN",Thursday,12,1 hr. 40 min.,15+ - Teens 15 or older,9.1,Everyday is extraordinary for five doctors and...,"Friendship, Romance, Life, Medical","Workplace, Strong Friendship, Best Friends, Mu...",Shin Won Ho,Lee Woo Jung,"Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Kim...","Egg Is Coming, CJ ENM",#4
4,My Mister,"Mar 21, 2018 - May 17, 2018",2018,tvN,"Wednesday, Thursday",16,1 hr. 17 min.,15+ - Teens 15 or older,9.1,Park Dong Hoon is a middle-aged engineer who i...,"Psychological, Life, Drama, Family","Age Gap, Nice Male Lead, Strong Female Lead, H...","Kim Won Suk, Kim Sang Woo",Park Hae Young,"Lee Sun Kyun, IU, Park Ho San, Song Sae Byuk, ...",Chorokbaem Media,#5


In [184]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [186]:
columns_to_drop = ['Aired Date', 'Aired On', 'Duration', 'Content Rating', 'Number of Episodes', 'Synopsis', 'Tags', 'Director', 'Screenwriter', 'Production companies']
df1 = df1.drop(columns=columns_to_drop)

In [270]:
df1.head()

,Year of release,Original Network,Rating,Genre,Cast,Rank,Cast_Similarity
Name,,,,,,,
Move to Heaven,2021,[Netflix],9.2,"[Life, Drama, Family]","Lee Je Hoon, Tang Jun Sang, Hong Seung Hee",1,False
Flower of Evil,2020,[tvN],9.1,"[Thriller, Romance, Crime, Melodrama]","Lee Joon Gi, Moon Chae Won, Jang Hee Jin",2,False
Hospital Playlist,2020,"[Netflix, tvN]",9.1,"[Friendship, Romance, Life, Medical]","Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho",3,False
Hospital Playlist 2,2021,"[Netflix, tvN]",9.1,"[Friendship, Romance, Life, Medical]","Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho",4,False
My Mister,2018,[tvN],9.1,"[Psychological, Life, Drama, Family]","Lee Sun Kyun, IU, Park Ho San",5,False


In [268]:
#df1['Genre'] = df1['Genre'].str.split(', ').apply(lambda genres: [genre.strip() for genre in genres])
#df1['Cast'] = df1['Cast'].apply(lambda x: ', '.join(x.split(', ')[:3]))
#df1.set_index('Name', inplace=True)
#df1['Original Network'] = df1['Original Network'].str.split(', ').apply(lambda networks: [network.strip() for network in networks])
#df1['Rank'] = df1['Rank'].str.extract('(\d+)').astype(int)

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
/var/folders/y5/qxtlt7t95bd97720vt1v56hc0000gn/T/ipykernel_95875/2414379497.py:5: SyntaxWarning: invalid escape sequence '\d'
  df1['Rank'] = df1['Rank'].str.extract('(\d+)').astype(int)


In [284]:
def jaccard_similarity(list1, list2):
    intersection = len(set(list1).intersection(set(list2)))
    union = len(set(list1).union(set(list2)))
    return intersection / union

def get_recommendations(kdrama_name, df, variables, num_recommendations=5):
    # Check if the K-Drama name is in the dataset
    if kdrama_name not in df.index:
        return "K-Drama not found."

    # Print the standardized variables for debugging
    #print("Variables for comparison:")
    #print(variables)

    # Separate numeric variables from non-numeric variables
    numeric_vars = df[variables].select_dtypes(include=['float64', 'int64']).columns.tolist()
    non_numeric_vars = [var for var in variables if var not in numeric_vars]

    # Print the numeric and non-numeric variables for debugging
    #print("Numeric variables:")
    #print(numeric_vars)
    #print("Non-numeric variables:")
    #print(non_numeric_vars)

    combined_similarity = pd.Series(0, index=df.index)

    # Calculate year similarity based on the absolute difference in years
    if 'Year of release' in variables:
        year_diff = df['Year of release'].apply(lambda x: abs(x - df.loc[kdrama_name, 'Year of release']))
        year_similarity = (year_diff.max() - year_diff) / year_diff.max()
        combined_similarity += year_similarity

    # Calculate Jaccard similarity for genres
    if 'Genre' in variables:
        genre_similarity = df['Genre'].apply(lambda x: jaccard_similarity(df.loc[kdrama_name, 'Genre'], x))
        combined_similarity += genre_similarity

    # Calculate Jaccard similarity for original networks
    if 'Original Network' in variables:
        network_similarity = df['Original Network'].apply(lambda x: jaccard_similarity(df.loc[kdrama_name, 'Original Network'], x))
        combined_similarity += network_similarity

    if numeric_vars:
        # Prepare the DataFrame for comparison using only numeric variables
        df_subset = df[numeric_vars]
        
        # Standardize the numeric variables
        scaler = StandardScaler()
        df_scaled = pd.DataFrame(scaler.fit_transform(df_subset), index=df_subset.index, columns=df_subset.columns)

        # Compute cosine similarity for numeric variables
        cosine_sim = cosine_similarity(df_scaled)
        cosine_sim_df = pd.DataFrame(cosine_sim, index=df_scaled.index, columns=df_scaled.index)
        combined_similarity += cosine_sim_df[kdrama_name]

    combined_similarity = combined_similarity.sort_values(ascending=False)

    # Get the top recommendations
    similar_kdramas = combined_similarity.index[combined_similarity.index != kdrama_name]
    recommendations = df.loc[similar_kdramas][:num_recommendations]
    output_variables = ['Rating', 'Rank'] + variables

    return recommendations[output_variables]

# Step 6: Ask the user for variables to use for similarity
variables = input('What similarities do you want to see between your K-Dramas? (comma-separated): ').split(', ')
variables = [var.strip() for var in variables]

# Step 7: Ask for the K-Drama name to use as the initial reference
kdrama_name = input('Enter K-Drama name: ')

# Step 8: Get the recommendations
recommendations = get_recommendations(kdrama_name, df1, variables)

# Step 9: Output the recommendations
print(f"Top {len(recommendations)} Recommendations:")
print(recommendations)

What similarities do you want to see between your K-Dramas? (comma-separated):  Genre, Rating, Cast
Enter K-Drama name:  My Mister


Top 5 Recommendations:
                 Rating  Rank  \
Name                            
Move to Heaven      9.2     1   
Navillera           9.0    16   
Dear My Friends     8.7    44   
SKY Castle          8.9    21   
Go Back Couple      8.6    83   

                                                             Genre  Rating  \
Name                                                                         
Move to Heaven                               [Life, Drama, Family]     9.2   
Navillera                        [Friendship, Life, Drama, Family]     9.0   
Dear My Friends                   [Life, Drama, Family, Melodrama]     8.7   
SKY Castle                 [Mystery, Psychological, Drama, Family]     8.9   
Go Back Couple   [Comedy, Romance, Life, School, Drama, Family,...     8.6   

                                                       Cast  
Name                                                         
Move to Heaven   Lee Je Hoon, Tang Jun Sang, Hong Seung Hee  
Navillera   

In [35]:
def get_recommendations(kdrama_name, pivot_table, num_recommendations=5):
    if kdrama_name not in pivot_table.index:
        return "K-Drama not found."
    kdrama_vector = pivot_table.loc[kdrama_name]
    similar_kdramas = pivot_table.corrwith(kdrama_vector)
    corr_df = pd.DataFrame(similar_kdramas, columns=['Correlation'])
    corr_df = corr_df.sort_values(by='Correlation', ascending=False)
    
    return corr_df.head(num_recommendations)

#test_kdrama = input('Enter K-Drama name: ')
#kdrama_vector = pt[test_kdrama].dropna()
#similar_kdramas = pt.corrwith(kdrama_vector)
#corr_df = pd.DataFrame(similar_kdramas,columns=['Correlation'])
#corr_df.head(10)
test_kdrama = input('Enter K-Drama name: ')
recommendations = get_recommendations(test_kdrama, pt)
print("Recommendations for {}: \n{}".format(test_kdrama, recommendations))

Enter K-Drama name:  My Mister


Recommendations for My Mister: 
        Correlation
Rating          NaN


In [280]:
variables = input('What similarities do you want to see between your K-Dramas? (comma-separated): ').split(', ')
variables = [var.strip() for var in variables]

# Step 6: Ask for the K-Drama name to use as the initial reference
kdrama_name = input('Enter K-Drama name: ')

# Function to calculate cast similarity
def cast_similarity(cast1, cast2):
    set1 = set(cast1.split(', '))
    set2 = set(cast2.split(', '))
    return len(set1.intersection(set2)) > 0

# Function to calculate Jaccard similarity for genres
def jaccard_similarity(list1, list2):
    intersection = len(set(list1).intersection(set(list2)))
    union = len(set(list1).union(set(list2)))
    return intersection / union

# Function to get recommendations
def get_recommendations(kdrama_name, df, variables, num_recommendations=5):
    # Check if the K-Drama name is in the dataset
    if kdrama_name not in df.index:
        return "K-Drama not found."

    # Separate numeric variables from non-numeric variables
    numeric_vars = df[variables].select_dtypes(include=['float64', 'int64']).columns.tolist()
    non_numeric_vars = [var for var in variables if var not in numeric_vars]

    # If no numeric variables are selected, use only non-numeric similarity
    if not numeric_vars:
        if 'Cast' in variables:
            cast1 = df.loc[kdrama_name, 'Cast']
            df['Cast_Similarity'] = df['Cast'].apply(lambda x: cast_similarity(cast1, x))
            recommendations = df[df['Cast_Similarity'] == True].drop(columns='Cast_Similarity')
            recommendations = recommendations[recommendations.index != kdrama_name].head(num_recommendations)
            print(f"Found {len(recommendations)} recommendations based on cast similarity.")
            return recommendations[variables]
        else:
            return "No numeric variables selected and 'Cast' is not among the selected variables."

    # Prepare the DataFrame for comparison using only numeric variables
    df_subset = df[numeric_vars]
    
    # Standardize the numeric variables
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_subset), index=df_subset.index, columns=df_subset.columns)

    # Compute cosine similarity for numeric variables
    cosine_sim = cosine_similarity(df_scaled)
    cosine_sim_df = pd.DataFrame(cosine_sim, index=df_scaled.index, columns=df_scaled.index)

    # Calculate Jaccard similarity for genres
    if 'Genre' in variables:
        genre_similarity = df['Genre'].apply(lambda x: jaccard_similarity(df.loc[kdrama_name, 'Genre'], x))
        genre_similarity_df = pd.DataFrame(genre_similarity, index=df.index, columns=['Genre_Similarity'])
    else:
        genre_similarity_df = pd.DataFrame(0, index=df.index, columns=['Genre_Similarity'])

    # Combine cosine similarity and genre similarity
    combined_similarity = 0.5 * cosine_sim_df[kdrama_name] + 0.5 * genre_similarity_df['Genre_Similarity']
    
    # If 'Cast' is among the variables, filter based on cast similarity
    if 'Cast' in variables:
        cast1 = df.loc[kdrama_name, 'Cast']
        df['Cast_Similarity'] = df['Cast'].apply(lambda x: cast_similarity(cast1, x))
        combined_similarity = combined_similarity[df['Cast_Similarity'] == True]
    
    combined_similarity = combined_similarity.sort_values(ascending=False)

    # Get the top recommendations
    similar_kdramas = combined_similarity.index[combined_similarity.index != kdrama_name]
    recommendations = df.loc[similar_kdramas][:num_recommendations]
    output_variables = ['Rating', 'Rank'] + variables

    return recommendations[output_variables]
    
    #print(f"Found {len(recommendations)} recommendations based on combined similarity.")
    #return recommendations[variables]

# Step 7: Get the recommendations
recommendations = get_recommendations(kdrama_name, df1, variables)

# Step 8: Output the recommendations
print(f"Top {len(recommendations)} Recommendations:")
print(recommendations)

What similarities do you want to see between your K-Dramas? (comma-separated):  a
Enter K-Drama name:  a


Top 18 Recommendations:
K-Drama not found.
